## 📥 1. Importación y Carga de los 4 Dataframes Limpios

Cargaremos los 4 dataframes desde los notebooks ya ejecutados. Esto requiere que los notebooks anteriores hayan sido ejecutados y los dataframes estén disponibles.

In [72]:
# Importaciones necesarias
import pandas as pd
from pathlib import Path

# Mostrar opciones de pandas para debugging
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 120)

print('Librerias cargadas correctamente.')


Librerias cargadas correctamente.


In [73]:
# Configuracion de rutas y artefactos intermedios (ETL)
PROJECT_DIR = Path.cwd()
DATA_RAW_DIR = PROJECT_DIR / 'db' / 'raw'
if not DATA_RAW_DIR.exists():
    DATA_RAW_DIR = PROJECT_DIR / 'db'

DATA_PROCESSED_DIR = PROJECT_DIR / 'db' / 'processed'
DATA_FINAL_DIR = PROJECT_DIR / 'db' / 'final'
DATA_PROCESSED_DIR.mkdir(parents=True, exist_ok=True)
DATA_FINAL_DIR.mkdir(parents=True, exist_ok=True)

DATA_SPECS = {
    'df_clientes_True': {
        'label': 'Clientes',
        'raw': 'clientes.xlsx',
        'processed': 'clientes_limpio.csv',
        'parse_dates': ['fecha_alta']
    },
    'df_Ventas_True': {
        'label': 'Ventas (cabecera)',
        'raw': 'ventas.xlsx',
        'processed': 'ventas_limpio.csv',
        'parse_dates': ['fecha']
    },
    'df_detalle_ventas_True': {
        'label': 'Detalle de ventas',
        'raw': 'detalle_ventas.xlsx',
        'processed': 'detalle_ventas_limpio.csv',
        'parse_dates': []
    },
    'df_productos_True': {
        'label': 'Productos',
        'raw': 'productos.xlsx',
        'processed': 'productos_limpio.csv',
        'parse_dates': []
    }
}

def _read_from_path(file_path, spec):
    suffix = file_path.suffix.lower()
    if suffix in {'.xlsx', '.xls'}:
        return pd.read_excel(file_path, parse_dates=spec.get('parse_dates') or [])
    if suffix == '.csv':
        return pd.read_csv(file_path, parse_dates=spec.get('parse_dates') or [])
    raise ValueError(f'No hay lector definido para {file_path}')

def load_clean_dataframe(var_name, spec):
    processed_path = DATA_PROCESSED_DIR / spec['processed']
    if processed_path.exists():
        df = _read_from_path(processed_path, spec)
        print(f"[OK ] {spec['label']:<24} -> {df.shape} | fuente: {processed_path.relative_to(PROJECT_DIR)}")
        return df
    raw_path = DATA_RAW_DIR / spec['raw']
    if not raw_path.exists():
        raise FileNotFoundError(f"No se encontro la fuente esperada: {raw_path}")
    df = _read_from_path(raw_path, spec)
    df.to_csv(processed_path, index=False, encoding='utf-8-sig')
    print(f"[NEW] {spec['label']:<24} -> {df.shape} | generado desde {raw_path.relative_to(PROJECT_DIR)}")
    print(f"      Artefacto intermedio guardado en {processed_path.relative_to(PROJECT_DIR)}")
    return df

loaded_frames = {var: load_clean_dataframe(var, spec) for var, spec in DATA_SPECS.items()}
globals().update(loaded_frames)
print("\nDataframes limpios disponibles en memoria.")


[OK ] Clientes                 -> (100, 5) | fuente: db\processed\clientes_limpio.csv
[OK ] Ventas (cabecera)        -> (120, 6) | fuente: db\processed\ventas_limpio.csv
[OK ] Detalle de ventas        -> (343, 6) | fuente: db\processed\detalle_ventas_limpio.csv
[OK ] Productos                -> (100, 4) | fuente: db\processed\productos_limpio.csv

Dataframes limpios disponibles en memoria.


In [74]:
# Funcion para verificar y reportar disponibilidad de dataframes
def check_dataframes():
    missing = []
    for var_name, spec in DATA_SPECS.items():
        processed_path = DATA_PROCESSED_DIR / spec['processed']
        artifact_status = 'OK' if processed_path.exists() else 'PENDIENTE'
        if var_name in globals():
            df_temp = globals()[var_name]
            print(f"OK  {spec['label']:30s} -> {df_temp.shape} | archivo: {artifact_status}")
        else:
            missing.append(spec)
            print(f"X   {spec['label']:30s} -> NO EN MEMORIA | archivo: {artifact_status}")
    if missing:
        print("\nAdvertencia: ejecuta los notebooks limp_y_trans_* para regenerar los archivos procesados.")
        return False
    print("\nTodos los dataframes limpios estan en memoria.")
    return True

# Verificar disponibilidad
all_available = check_dataframes()


OK  Clientes                       -> (100, 5) | archivo: OK
OK  Ventas (cabecera)              -> (120, 6) | archivo: OK
OK  Detalle de ventas              -> (343, 6) | archivo: OK
OK  Productos                      -> (100, 4) | archivo: OK

Todos los dataframes limpios estan en memoria.


## 📊 2. Inspección de Estructura y Claves

Antes de realizar los merges, inspeccionaremos cada dataframe para identificar correctamente las claves primarias (PK) y foráneas (FK).

In [75]:
if all_available:
    print('='*100)
    print('📋 ESTRUCTURA DE DATAFRAMES')
    print('='*100)
    
    print('\n1. df_clientes_True (PK: id_cliente)')
    print('-'*80)
    print(df_clientes_True.head(0))
    print(f'Columnas: {list(df_clientes_True.columns)}')
    print(f'Registros: {len(df_clientes_True)}')
    
    print('\n2. df_Ventas_True (PK: id_venta | FK: id_cliente)')
    print('-'*80)
    print(df_Ventas_True.head(0))
    print(f'Columnas: {list(df_Ventas_True.columns)}')
    print(f'Registros: {len(df_Ventas_True)}')
    
    print('\n3. df_detalle_ventas_True (PK: id_detalle | FK: id_venta, id_producto)')
    print('-'*80)
    print(df_detalle_ventas_True.head(0))
    print(f'Columnas: {list(df_detalle_ventas_True.columns)}')
    print(f'Registros: {len(df_detalle_ventas_True)}')
    
    print('\n4. df_productos_True (PK: id_producto)')
    print('-'*80)
    print(df_productos_True.head(0))
    print(f'Columnas: {list(df_productos_True.columns)}')
    print(f'Registros: {len(df_productos_True)}')

📋 ESTRUCTURA DE DATAFRAMES

1. df_clientes_True (PK: id_cliente)
--------------------------------------------------------------------------------
Empty DataFrame
Columns: [id_cliente, nombre_cliente, email, ciudad, fecha_alta]
Index: []
Columnas: ['id_cliente', 'nombre_cliente', 'email', 'ciudad', 'fecha_alta']
Registros: 100

2. df_Ventas_True (PK: id_venta | FK: id_cliente)
--------------------------------------------------------------------------------
Empty DataFrame
Columns: [id_venta, fecha, id_cliente, nombre_cliente, email, medio_pago]
Index: []
Columnas: ['id_venta', 'fecha', 'id_cliente', 'nombre_cliente', 'email', 'medio_pago']
Registros: 120

3. df_detalle_ventas_True (PK: id_detalle | FK: id_venta, id_producto)
--------------------------------------------------------------------------------
Empty DataFrame
Columns: [id_venta, id_producto, nombre_producto, cantidad, precio_unitario, importe]
Index: []
Columnas: ['id_venta', 'id_producto', 'nombre_producto', 'cantidad', 'pre

## 🔗 3. Preparación: Renombrado de Claves Foráneas

En la tabla de detalle de ventas, si existe un nombre diferente para la FK de producto (ej: FK_producto), lo renombramos a `id_producto` para que el merge sea unívoco.

In [76]:
if all_available:
    print('='*100)
    print('🔧 PREPARACIÓN: Validación y renombrado de columnas')
    print('='*100)
    
    # Verificar y renombrar columna de producto en detalle_ventas si es necesario
    if 'FK_producto' in df_detalle_ventas_True.columns:
        print('\n✓ Renombrando FK_producto → id_producto en df_detalle_ventas_True')
        df_detalle_ventas_True.rename(columns={'FK_producto': 'id_producto'}, inplace=True)
    elif 'id_producto' not in df_detalle_ventas_True.columns:
        print('\n⚠️  Aviso: No se encontró FK_producto ni id_producto en detalle_ventas')
        print(f'   Columnas disponibles: {list(df_detalle_ventas_True.columns)}')
    else:
        print('\n✓ Columna id_producto ya existe en df_detalle_ventas_True')
    
    # Validar que todos los FK referencias existan en las PK correspondientes
    print('\n' + '-'*80)
    print('📌 Validación de Integridad Referencial:')
    print('-'*80)
    
    # FK id_cliente en ventas
    clientes_en_ventas = set(df_Ventas_True['id_cliente'].dropna().unique())
    clientes_existentes = set(df_clientes_True['id_cliente'].unique())
    clientes_huerfanos = clientes_en_ventas - clientes_existentes
    print(f'\n✓ FK id_cliente en ventas:',)
    if not clientes_huerfanos:
        print(f'OK ({len(clientes_en_ventas)} referencias válidas)')
    else:
        print(f'INCONSISTENCIA: {len(clientes_huerfanos)} huérfanos')
    
    # FK id_venta en detalle
    ventas_en_detalle = set(df_detalle_ventas_True['id_venta'].dropna().unique())
    ventas_existentes = set(df_Ventas_True['id_venta'].unique())
    ventas_huerfanas = ventas_en_detalle - ventas_existentes
    print(f'✓ FK id_venta en detalle:',)
    if not ventas_huerfanas:
        print(f'OK ({len(ventas_en_detalle)} referencias válidas)')
    else:
        print(f'INCONSISTENCIA: {len(ventas_huerfanas)} huérfanos')
    
    # FK id_producto en detalle
    productos_en_detalle = set(df_detalle_ventas_True['id_producto'].dropna().unique())
    productos_existentes = set(df_productos_True['id_producto'].unique())
    productos_huerfanos = productos_en_detalle - productos_existentes
    print(f'✓ FK id_producto en detalle:',)
    if not productos_huerfanos:
        print(f'OK ({len(productos_en_detalle)} referencias válidas)')
    else:
        print(f'INCONSISTENCIA: {len(productos_huerfanos)} huérfanos')

🔧 PREPARACIÓN: Validación y renombrado de columnas

✓ Columna id_producto ya existe en df_detalle_ventas_True

--------------------------------------------------------------------------------
📌 Validación de Integridad Referencial:
--------------------------------------------------------------------------------

✓ FK id_cliente en ventas:
OK (67 referencias válidas)
✓ FK id_venta en detalle:
OK (120 referencias válidas)
✓ FK id_producto en detalle:
OK (95 referencias válidas)


## 🔀 4. Merge de Tablas - Consolidación de Base Final

Realizamos los merges en orden siguiendo la estructura de relaciones:
1. `df_clientes_True` ← merge ← `df_Ventas_True` (on=id_cliente)
2. Resultado ← merge ← `df_detalle_ventas_True` (on=id_venta)
3. Resultado ← merge ← `df_productos_True` (on=id_producto)

In [77]:
if all_available:
    print('='*100)
    print('🔀 MERGE - Consolidación de Base Final')
    print('='*100)
    
    print('\n1️⃣  Merge: df_clientes_True + df_Ventas_True (on=id_cliente)')
    print('-'*80)
    base_consolidada = df_clientes_True.merge(
        df_Ventas_True,
        on='id_cliente',
        how='inner',  # inner join para evitar clientes sin ventas
        validate='1:m'  # 1 cliente : muchas ventas
    )
    print(f'✓ Dimensiones resultado: {base_consolidada.shape}')
    print(f'  - Registros: {len(base_consolidada)}')
    print(f'  - Columnas: {len(base_consolidada.columns)}')
    
    print('\n2️⃣  Merge: resultado + df_detalle_ventas_True (on=id_venta)')
    print('-'*80)
    base_consolidada = base_consolidada.merge(
        df_detalle_ventas_True,
        on='id_venta',
        how='inner',  # inner join para que todo tenga detalle
        validate='m:m'  # muchas ventas : muchos detalles
    )
    print(f'✓ Dimensiones resultado: {base_consolidada.shape}')
    print(f'  - Registros: {len(base_consolidada)}')
    print(f'  - Columnas: {len(base_consolidada.columns)}')
    
    print('\n3️⃣  Merge: resultado + df_productos_True (on=id_producto)')
    print('-'*80)
    base_consolidada = base_consolidada.merge(
        df_productos_True,
        on='id_producto',
        how='inner',  # inner join para evitar productos sin venta
        validate='m:1'  # muchos detalles : 1 producto
    )
    print(f'✓ Dimensiones resultado: {base_consolidada.shape}')
    print(f'  - Registros: {len(base_consolidada)}')
    print(f'  - Columnas: {len(base_consolidada.columns)}')
    
    print('\n✨ Merge completado exitosamente!')

🔀 MERGE - Consolidación de Base Final

1️⃣  Merge: df_clientes_True + df_Ventas_True (on=id_cliente)
--------------------------------------------------------------------------------
✓ Dimensiones resultado: (120, 10)
  - Registros: 120
  - Columnas: 10

2️⃣  Merge: resultado + df_detalle_ventas_True (on=id_venta)
--------------------------------------------------------------------------------
✓ Dimensiones resultado: (343, 15)
  - Registros: 343
  - Columnas: 15

3️⃣  Merge: resultado + df_productos_True (on=id_producto)
--------------------------------------------------------------------------------
✓ Dimensiones resultado: (343, 18)
  - Registros: 343
  - Columnas: 18

✨ Merge completado exitosamente!
✓ Dimensiones resultado: (343, 18)
  - Registros: 343
  - Columnas: 18

✨ Merge completado exitosamente!


## 📊 5. Inspección de la Base Consolidada

In [78]:
if 'base_consolidada' in locals():
    print('='*100)
    print('📋 INSPECCIÓN - Base Consolidada')
    print('='*100)
    
    print('\nPrimeras 5 filas:')
    print('-'*80)
    print(base_consolidada.head())
    
    print('\n\nÚltimas 5 filas:')
    print('-'*80)
    print(base_consolidada.tail())
    
    print('\n\nTipos de datos:')
    print('-'*80)
    print(base_consolidada.dtypes)
    
    print('\n\nValores nulos por columna:')
    print('-'*80)
    nulls = base_consolidada.isnull().sum()
    if nulls.sum() == 0:
        print('✓ NO hay valores nulos en la base consolidada')
    else:
        print(nulls[nulls > 0])
    
    print('\n\nEstadísticas generales:')
    print('-'*80)
    print(f'Dimensiones: {base_consolidada.shape}')
    print(f'Memoria usada: {base_consolidada.memory_usage(deep=True).sum() / (1024**2):.2f} MB')
    print(f'Clientes únicos: {base_consolidada["id_cliente"].nunique()}')
    print(f'Ventas (id_venta) únicas: {base_consolidada["id_venta"].nunique()}')
    print(f'Detalles de venta únicos: {base_consolidada["id_detalle"].nunique() if "id_detalle" in base_consolidada.columns else "N/A"}')
    print(f'Productos únicos: {base_consolidada["id_producto"].nunique()}')

📋 INSPECCIÓN - Base Consolidada

Primeras 5 filas:
--------------------------------------------------------------------------------
   id_cliente nombre_cliente_x                 email_x      ciudad fecha_alta  id_venta      fecha nombre_cliente_y  \
0           1    Mariana Lopez  mariana.lopez@mail.com  Carlos Paz 2023-01-01        54 2024-03-26    Mariana Lopez   
1           1    Mariana Lopez  mariana.lopez@mail.com  Carlos Paz 2023-01-01        54 2024-03-26    Mariana Lopez   
2           1    Mariana Lopez  mariana.lopez@mail.com  Carlos Paz 2023-01-01        54 2024-03-26    Mariana Lopez   
3           1    Mariana Lopez  mariana.lopez@mail.com  Carlos Paz 2023-01-01        54 2024-03-26    Mariana Lopez   
4           1    Mariana Lopez  mariana.lopez@mail.com  Carlos Paz 2023-01-01       105 2024-02-06    Mariana Lopez   

                  email_y     medio_pago  id_producto       nombre_producto_x  cantidad  precio_unitario_x  importe  \
0  mariana.lopez@mail.com        t

## 🧹 6. Preprocesamiento y Limpieza Adicional

En esta sección aplicamos transformaciones finales para garantizar que la base esté lista para análisis.

In [79]:
if 'base_consolidada' in locals():
    print('='*100)
    print('🧹 PREPROCESAMIENTO - Limpieza y Transformaciones Finales')
    print('='*100)
    
    # Crear copia para no modificar original hasta validar
    base_final = base_consolidada.copy()
    
    # 1. Detectar y manejar duplicados
    print('\n1️⃣  Detección de duplicados')
    print('-'*80)
    dup_count = base_final.duplicated().sum()
    print(f'Filas duplicadas: {dup_count}')
    if dup_count > 0:
        print(f'  (Eliminando {dup_count} filas duplicadas)')
        base_final = base_final.drop_duplicates()
        print(f'  Nuevo tamaño: {base_final.shape}')
    
    # 2. Renombrar columnas para mayor claridad y evitar colisiones
    print('\n2️⃣  Estandarización de nombres de columnas')
    print('-'*80)
    rename_map = {}
    for col in base_final.columns:
        # Aquí puedes definir transformaciones de nombres si es necesario
        # Ejemplo: renombrar columnas que tengan sufijos _x o _y
        if col.endswith('_x'):
            rename_map[col] = col[:-2]
        elif col.endswith('_y'):
            rename_map[col] = col[:-2]
    
    if rename_map:
        print(f'Columnas renombradas:')
        for old, new in rename_map.items():
            print(f'  {old:30s} → {new}')
        base_final.rename(columns=rename_map, inplace=True)
    else:
        print('✓ No hay columnas con sufijos _x o _y')
    

    # 2b. Eliminar columnas duplicadas post-merge
    print('\n2b  Columnas duplicadas tras el merge')
    print('-'*80)
    duplicated_mask = base_final.columns.duplicated()
    if duplicated_mask.any():
        dup_names = base_final.columns[duplicated_mask]
        dup_list = sorted({str(name) for name in dup_names})
        base_final = base_final.loc[:, ~duplicated_mask]
        print(f" Columnas eliminadas: {dup_list}")
    else:
        print(' No se detectaron columnas duplicadas')

    # 3. Reordenar columnas de forma lógica (primero IDs, luego datos descriptivos)
    print('\n3️⃣  Reorganización de columnas (orden lógico)')
    print('-'*80)
    id_cols = [c for c in base_final.columns if 'id' in c.lower()]
    date_cols = [c for c in base_final.columns if 'fecha' in c.lower() or 'date' in c.lower()]
    money_cols = [c for c in base_final.columns if any(x in c.lower() for x in ['precio', 'importe', 'monto', 'total'])]
    other_cols = [c for c in base_final.columns if c not in id_cols + date_cols + money_cols]
    
    new_order = id_cols + date_cols + other_cols + money_cols
    base_final = base_final[[c for c in new_order if c in base_final.columns]]
    print(f'✓ Columnas reordenadas: {len(new_order)} columnas')
    print(f'  - IDs (PK/FK): {len(id_cols)}')
    print(f'  - Fechas: {len(date_cols)}')
    print(f'  - Dinero (precios/importes): {len(money_cols)}')
    print(f'  - Otros: {len(other_cols)}')
    
    # 4. Tipos de datos finales
    print('\n4️⃣  Validación de tipos de datos')
    print('-'*80)
    print(base_final.dtypes)
    
    print('\n✨ Preprocesamiento completado')

🧹 PREPROCESAMIENTO - Limpieza y Transformaciones Finales

1️⃣  Detección de duplicados
--------------------------------------------------------------------------------
Filas duplicadas: 0

2️⃣  Estandarización de nombres de columnas
--------------------------------------------------------------------------------
Columnas renombradas:
  nombre_cliente_x               → nombre_cliente
  email_x                        → email
  nombre_cliente_y               → nombre_cliente
  email_y                        → email
  nombre_producto_x              → nombre_producto
  precio_unitario_x              → precio_unitario
  nombre_producto_y              → nombre_producto
  precio_unitario_y              → precio_unitario

2b  Columnas duplicadas tras el merge
--------------------------------------------------------------------------------
 Columnas eliminadas: ['email', 'nombre_cliente', 'nombre_producto', 'precio_unitario']

3️⃣  Reorganización de columnas (orden lógico)
----------------------

## 📄 7. Exportación a CSV

In [80]:
if 'base_final' in locals():
    print('='*100)
    print(' EXPORTACION - Base Final a CSV')
    print('='*100)
    
    # Definir ruta de exportacion
    export_dir = DATA_FINAL_DIR
    export_dir.mkdir(parents=True, exist_ok=True)
    export_file = export_dir / 'Base_Final_Aurelion.csv'
    
    # Exportar a CSV
    print(f"\n Exportando a: {export_file}")
    base_final.to_csv(export_file, index=False, encoding='utf-8-sig')
    
    # Validar exportacion
    if export_file.exists():
        file_size_mb = export_file.stat().st_size / (1024**2)
        print(' Exportacion exitosa')
        print(' Detalles del archivo:')
        print(f"   - Nombre: {export_file.name}")
        print(f"   - Ruta: {export_file.resolve()}")
        print(f"   - Tamano: {file_size_mb:.2f} MB")
        print(f"   - Registros: {len(base_final):,}")
        print(f"   - Columnas: {len(base_final.columns)}")
    else:
        print(' Error: El archivo no se creo correctamente')
    
    print(' Consolidacion completada exitosamente')

 EXPORTACION - Base Final a CSV

 Exportando a: c:\Users\Asus\Desktop\Augusto Villegas - Proyecto Aurelion\db\final\Base_Final_Aurelion.csv
 Exportacion exitosa
 Detalles del archivo:
   - Nombre: Base_Final_Aurelion.csv
   - Ruta: C:\Users\Asus\Desktop\Augusto Villegas - Proyecto Aurelion\db\final\Base_Final_Aurelion.csv
   - Tamano: 0.04 MB
   - Registros: 343
   - Columnas: 14
 Consolidacion completada exitosamente


## 📋 8. Resumen Final

In [81]:
print('='*100)
print(' RESUMEN FINAL - CONSOLIDACION PROYECTO AURELION')
print('='*100)

print(' Objetivo cumplido:')
print('    Se integraron los 4 dataframes limpios (clientes, ventas, detalle_ventas, productos)')
print('    Se aplicaron relaciones de clave primaria (PK) y foranea (FK)')
print('    Se valido integridad referencial')
print('    Se realizo preprocesamiento y limpieza adicional')
print('    Se exporto base consolidada a CSV')

print(' Estadisticas finales:')
if 'base_final' in locals():
    export_dest = DATA_FINAL_DIR / 'Base_Final_Aurelion.csv'
    print(f"   - Registros en base final: {len(base_final):,}")
    print(f"   - Columnas integradas: {len(base_final.columns)}")
    print(f"   - Memoria usada: {base_final.memory_usage(deep=True).sum() / (1024**2):.2f} MB")
    print(f"   - Ubicacion archivo CSV: {export_dest}")

print(' Proximos pasos:')
print('   1. Usar Base_Final_Aurelion.csv para analisis avanzados')
print('   2. Crear visualizaciones estrategicas')
print('   3. Desarrollar modelos predictivos o de segmentacion')
print('   4. Generar reportes ejecutivos')

print(' Consolidacion finalizada correctamente')


 RESUMEN FINAL - CONSOLIDACION PROYECTO AURELION
 Objetivo cumplido:
    Se integraron los 4 dataframes limpios (clientes, ventas, detalle_ventas, productos)
    Se aplicaron relaciones de clave primaria (PK) y foranea (FK)
    Se valido integridad referencial
    Se realizo preprocesamiento y limpieza adicional
    Se exporto base consolidada a CSV
 Estadisticas finales:
   - Registros en base final: 343
   - Columnas integradas: 14
   - Memoria usada: 0.14 MB
   - Ubicacion archivo CSV: c:\Users\Asus\Desktop\Augusto Villegas - Proyecto Aurelion\db\final\Base_Final_Aurelion.csv
 Proximos pasos:
   1. Usar Base_Final_Aurelion.csv para analisis avanzados
   2. Crear visualizaciones estrategicas
   3. Desarrollar modelos predictivos o de segmentacion
   4. Generar reportes ejecutivos
 Consolidacion finalizada correctamente
